
# Transfer Learning con ResNet18, DenseNet121 y VGG16

In [1]:
# %pip install medmnist

In [8]:

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision.models import ResNet18_Weights, DenseNet121_Weights, VGG16_Weights
import torchvision.transforms as transforms
import medmnist
from medmnist import INFO

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Cargar dataset
data_flag = 'breastmnist'
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

train_dataset = DataClass(split='train', transform=transform, download=True)
val_dataset = DataClass(split='val', transform=transform, download=True)
test_dataset = DataClass(split='test', transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [9]:
def get_model(name, num_classes=1):
    if name == 'resnet':
        from torchvision.models import resnet18, ResNet18_Weights
        model = resnet18(weights=ResNet18_Weights.DEFAULT)
        in_features = model.fc.in_features
        model.fc = nn.Sequential(nn.Linear(in_features, num_classes), nn.Sigmoid())

    elif name == 'densenet':
        from torchvision.models import densenet121, DenseNet121_Weights
        model = densenet121(weights=DenseNet121_Weights.DEFAULT)
        in_features = model.classifier.in_features
        model.classifier = nn.Sequential(nn.Linear(in_features, num_classes), nn.Sigmoid())

    elif name == 'vgg':
        from torchvision.models import vgg16, VGG16_Weights
        model = vgg16(weights=VGG16_Weights.DEFAULT)
        in_features = model.classifier[6].in_features
        model.classifier[6] = nn.Sequential(nn.Linear(in_features, num_classes), nn.Sigmoid())

    # Congelar todas las capas excepto la cabeza (fc)
    for param in model.parameters():
        param.requires_grad = False

    # Solo entrenar la cabeza
    if name == 'resnet':
        for param in model.fc.parameters():
            param.requires_grad = True
    elif name == 'densenet':
        for param in model.classifier.parameters():
            param.requires_grad = True
    elif name == 'vgg':
        for param in model.classifier[6].parameters():
            param.requires_grad = True

    return model

In [10]:
def evaluate(model, loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            labels = labels.float().reshape(-1, 1)
            outputs = model(imgs)
            y_true.extend(labels.numpy())
            y_pred.extend(outputs.numpy())
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred_class = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_true, y_pred_class)
    return acc

def train_and_evaluate(model, name, epochs=5):
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

    best_val_acc = 0
    best_state = None

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for imgs, labels in train_loader:
            labels = labels.float().reshape(-1, 1)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            preds = (outputs > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        train_acc = correct / total

        val_acc = evaluate(model, val_loader)
        print(f"[{name}] Epoch {epoch+1}: Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

        # Guardar el mejor modelo según validación
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = model.state_dict()

    # Restaurar el mejor modelo
    if best_state is not None:
        model.load_state_dict(best_state)

    # Evaluación final en test
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            labels = labels.float().reshape(-1, 1)
            outputs = model(imgs)
            y_true.extend(labels.numpy())
            y_pred.extend(outputs.numpy())
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred_class = (y_pred > 0.5).astype(int)

    acc = accuracy_score(y_true, y_pred_class)
    f1 = f1_score(y_true, y_pred_class)
    prec = precision_score(y_true, y_pred_class)
    rec = recall_score(y_true, y_pred_class)
    roc_auc = roc_auc_score(y_true, y_pred)

    print(f"\n{name.upper()} Results (Best Val Acc: {best_val_acc:.4f}):")
    print(f"Test Accuracy:  {acc:.4f}")
    print(f"F1 Score:      {f1:.4f}")
    print(f"Precision:     {prec:.4f}")
    print(f"Recall:        {rec:.4f}")
    print(f"AUC-ROC:       {roc_auc:.4f}")

In [11]:
for name in ['resnet', 'densenet', 'vgg']:
    print(f"\nEntrenando modelo: {name.upper()}")
    model = get_model(name)
    train_and_evaluate(model, name, epochs=5)
    # Guardar los modelos entrenados
    torch.save(model.state_dict(), f'{name}_model.pth')


Entrenando modelo: RESNET
[resnet] Epoch 1: Loss=10.2483, Train Acc=0.7198, Val Acc=0.7308
[resnet] Epoch 2: Loss=10.0558, Train Acc=0.7344, Val Acc=0.7051
[resnet] Epoch 3: Loss=9.1579, Train Acc=0.7527, Val Acc=0.7436
[resnet] Epoch 4: Loss=8.8068, Train Acc=0.7509, Val Acc=0.7308
[resnet] Epoch 5: Loss=8.7060, Train Acc=0.7473, Val Acc=0.7179

RESNET Results (Best Val Acc: 0.7436):
Test Accuracy:  0.7436
F1 Score:      0.8214
Precision:     0.8364
Recall:        0.8070
AUC-ROC:       0.7464

Entrenando modelo: DENSENET
[densenet] Epoch 1: Loss=10.6142, Train Acc=0.7344, Val Acc=0.7308
[densenet] Epoch 2: Loss=9.8843, Train Acc=0.7344, Val Acc=0.7179
[densenet] Epoch 3: Loss=9.2333, Train Acc=0.7363, Val Acc=0.7308
[densenet] Epoch 4: Loss=9.1881, Train Acc=0.7436, Val Acc=0.7436
[densenet] Epoch 5: Loss=9.4580, Train Acc=0.7674, Val Acc=0.8205

DENSENET Results (Best Val Acc: 0.8205):
Test Accuracy:  0.7885
F1 Score:      0.8696
Precision:     0.7914
Recall:        0.9649
AUC-ROC: 